In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
csv_gt_path = '--path ground truth.csv--'
csv_gt = pd.read_csv(csv_gt_path)
csv_p = pd.read_csv("--output of classifier.csv--")

In [ ]:
def IOU(bb, bb_gt1, bb_gt2=None):
    """
    INPUTS
        bb -> bounding box
        bb_gt1 -> first ground-truth bounding box
        bb_gt2 -> second ground-truth bounding box (if exists)
    RETURNS
        Intersection Over Union for the detected bounding box and the ground truth box
    """
    l = float(bb[0])
    t = float(bb[1])
    r = float(bb[2])
    b = float(bb[3])
    l1 = float(bb_gt1[0])
    t1 = float(bb_gt1[1])
    r1 = float(bb_gt1[2])
    b1 = float(bb_gt1[3])

    ru1 = min(r,r1)
    lu1 = max(l,l1)
    tu1 = min(t,t1)
    bu1 = max(b,b1)
    if (ru1>lu1) and (tu1>bu1):
        area_of_overlap_1 = (ru1-lu1) * (tu1-bu1)
        area_of_union_1 = (t-b)*(r-l) + (t1-b1)*(r1-l1) - area_of_overlap_1
        iou_1 = area_of_overlap_1/area_of_union_1
    else:
        iou_1 = 0

    if bb_gt2 != None:
        r2 = float(bb_gt2[0])
        t2 = float(bb_gt2[1])
        l2 = float(bb_gt2[2])
        b2 = float(bb_gt2[3])
        
        ru2 = min(r,r2)
        lu2 = max(l,l2)
        tu2 = min(t,t2)
        bu2 = max(b,b2)
        if (ru2>lu2) and (tu2>bu2):
            area_of_overlap_2 = (ru2-lu2) * (tu2-bu2)
            area_of_union_2 = (t-b)*(r-l) + (t2-b2)*(r2-l2) - area_of_overlap_2
            iou_2 = area_of_overlap_2/area_of_union_2
        else:
            iou_2 = 0    
        iou = max(iou_1, iou_2)
    else:
        iou = iou_1

    return iou

In [ ]:
def precision(tp,fp):
    p = tp/(tp+fp)
    return p

def recall(tp,fn):
    r = tp/(tp+fn)
    return r

def acc(tp,tn,fp,fn):
    a = (tp+tn)/(tp+fp+tn+fn)        
    return a

def f1(tp,tn,fp,fn):
    p = precision(tp,fp)
    r = recall(tp,fn)
    f = 2*p*r/(p+r)
    return f    

In [ ]:
def get_bbox(row):
    img_size = 1024
    new_img_size = 128

    x = row[0]
    y = row[1]
    w = row[2]
    h = row[3]

    rw = w/img_size
    rh = h/img_size
    rx = x/img_size + rw/2
    ry = y/img_size + rw/2
    
    l = int((rx-rw/2)*new_img_size)
    t = int((ry-rh/2)*new_img_size+rh*new_img_size)
    r = int(l + rw*new_img_size)
    b = int((ry-rh/2)*new_img_size)

    bb = [l,t,r,b]
    return bb

In [ ]:
def get_ground_truth(patient_id):
    label = csv_gt.query('patientId == "' + patient_id + '"').values[0:2]
    if label[0][5] == 0:
        return 0
    else:
        if len(label) < 2:
            bbox = get_bbox(label[0][1:5])
            return bbox
        else:   #if there is more than one bounding box
            l1 = label[0]
            l2 = label[1]
            bbox1 = get_bbox(l1[1:5])   
            bbox2 = get_bbox(l2[1:5])
            bbox = [bbox1, bbox2]
            return bbox


In [ ]:
t_p = 0
t_n = 0
f_p = 0
f_n = 0
iou = []

In [ ]:
last_id = 'none'

for i, row in csv_p.iterrows():
    if i == 0:
        pat_id = row['patientId'][:-1]
        bbox = row['bbox']
        gt_bbox = get_ground_truth(pat_id)
        if bbox == 'none':
            if gt_bbox == 0:
                t_n = t_n + 1
            else:
                f_n = f_n + 1
        else:
            bb = bbox.split(' ')
            if gt_bbox == 0:
                f_p = f_p + 1
            else:
                t_p = t_p + 1 
                if len(gt_bbox) == 4:
                    iou = iou + [IOU(bb, gt_bbox)]
                else:
                    iou = iou + [IOU(bb, gt_bbox[0], gt_bbox[1])]
    else:    
        pat_id = row['patientId'][:-1]
        bbox = row['bbox']
        gt_bbox = get_ground_truth(pat_id)
        
        if last_id != pat_id:
            if bbox == 'none':
                if gt_bbox == 0:
                    t_n = t_n + 1
                else:
                    f_n = f_n + 1
            else:
                bb = bbox.split(' ')
                if gt_bbox == 0:
                    f_p = f_p + 1
                else:
                    t_p = t_p + 1 
                    if len(gt_bbox) == 4:
                        iou = iou + [IOU(bb, gt_bbox)]
                    else:
                        iou = iou + [IOU(bb, gt_bbox[0], gt_bbox[1])]  
        else:
            if gt_bbox != 0:
                if len(gt_bbox) == 4:
                    iou = iou + [IOU(bb, gt_bbox)]
                else:
                    iou = iou + [IOU(bb, gt_bbox[0], gt_bbox[1])]
    last_id = pat_id    

In [ ]:
print('True positive: {}'.format(t_p))
print('True negative: {}'.format(t_n))
print('False positive: {}'.format(f_p))
print('False negative: {}'.format(f_n))

In [ ]:
mean_iou = np.mean(iou)
print("Mean IOU: {}".format(round(mean_iou,3)))

In [ ]:
p = precision(t_p, f_p)
r = recall(t_p, f_n)
a = acc(t_p, t_n, f_p, f_n)
f = f1(t_p, t_n, f_p, f_n)

In [ ]:
print('Precision: {}'.format(round(p,2)))
print('Recall: {}'.format(round(r,2)))
print('Accuracy: {}'.format(round(a,2)))
print('F1-score: {}'.format(round(f,2)))